# Test of our results

In this file, we demontsrate how well our model performs compared to the baseline and we find the best parameters for our preprocessing algorithm.

In [ ]:
import pandas as pd
from pipeline import get_full_pipeline_with_model
from baseline_pipeline import get_baseline_full_pipeline
from sklearn.model_selection import train_test_split

data = pd.read_csv('CarsData.csv')

full_pipeline_with_model = get_full_pipeline_with_model(data, 'price')
baseline_full_pipeline = get_baseline_full_pipeline(data, 'price')

y = data.price
X = data.drop('price', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Here we search for the best parameters:

In [ ]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")


grid_params = [
    {
        'full_pipeline__select_from_model__threshold' : [50, 100, 200],
        'full_pipeline__feature_extraction__method': ['PCA'],
        'full_pipeline__feature_extraction__n_components': [0.9, 0.95, 0.99]
    },
    {
        'full_pipeline__select_from_model__threshold' : [50, 100, 200],
        'full_pipeline__feature_extraction__method': ['LDA'],
        'full_pipeline__feature_extraction__n_components': [15, 30, 50]
    }
]

grid = GridSearchCV(full_pipeline_with_model, grid_params, verbose=3, cv = 3, scoring = 'neg_mean_squared_error')
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
model = grid.best_estimator_
model

Here we compute the results (error) after preprocessing:

In [ ]:
pip = full_pipeline_with_model.fit(X_train, y_train)

In [ ]:
import numpy as np
predicted = model.predict(X_test)
np.sqrt(np.sum((predicted -  y_test) ** 2) / predicted.shape[0])

Here we compute the results without preprocessing:

In [ ]:
sim = baseline_full_pipeline
sim.fit(X_train, y_train)

In [ ]:
import numpy as np
predicted = sim.predict(X_test)
np.sqrt(np.sum((predicted -  y_test) ** 2) / predicted.shape[0])

We can see a clear improvement.